In [50]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

len4 blp

In [51]:
count=0
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)
# len4_ndb is left joined so delete duplicated rows
data=data.drop_duplicates(subset=["id","year","総計"])
# drop never treated group
data=data.dropna(subset=["Approved"])
data=data.dropna(subset=["総計"])
data.head()
data.shape

C:\Users\hakos\AppData\Local\Temp\ipykernel_17312\2302481980.py:2: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)


(29653, 18)

In [52]:
duplicate_rows = data[data.duplicated(subset=['year', 'id', '総計'])]

# 重複行の数を表示
print("重複行の数:", len(duplicate_rows))

# 重複行のデータを表示
print(duplicate_rows)

重複行の数: 0
Empty DataFrame
Columns: [薬効分類, 薬効分類名称, 医薬品コード, 医薬品名, 薬価基準収載医薬品コード, 薬価, 後発品区分, 総計, year, in_hospital, oral, id_l4, Approved, KEGG, Ingredient, variable, id, revenue]
Index: []


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [53]:
data["elasped"]=data["year"]-data["Approved"]
#　不要なところを除外
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","id"])[sumcolumns].sum().reset_index()
# save temporarily
colnames=dummies.columns.to_list()
sums=data.groupby(["year","id"])[sumcolumns].sum().reset_index()
# save temporarily
dummy_cols=dummies.columns.to_list()

# sum by generic
sum_generic=data.groupby(["year","id","後発品区分"])[sumcolumns].sum().reset_index()

if count:
    # count ver.
    aggregated_df = pd.concat([data.groupby(['year', 'id']).first().reset_index()[['year', 'id']], 
      dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index(drop=True)
else:
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id']).first().reset_index()[['year', 'id']],
        dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int)).reset_index(drop=True)
    ], axis=1).reset_index(drop=True)
# # sumをとったところを追加する

aggregated_df[sumcolumns]=sums[sumcolumns]
aggregated_df.rename(columns={"総計":"sum_quantity","revenue":"sum_revenue"},inplace=True)
aggregated_df["mean_price"]=aggregated_df["sum_revenue"]/aggregated_df["sum_quantity"]
aggregated_df["lag_mean_price"]=aggregated_df.groupby(["id"])["mean_price"].shift(1)
# lag_sum_quantity
aggregated_df["lag_sum_quantity"]=aggregated_df.groupby(["id"])["sum_quantity"].shift(1)
aggregated_df.shape

(688, 52)

In [54]:
sum_generic["後発品区分"]=sum_generic["後発品区分"].astype("int")
sum_generic=sum_generic.pivot_table(index=["year","id"],columns="後発品区分",values=sumcolumns,aggfunc="sum").reset_index()
sum_generic.columns = [f"{col[0]}{col[1]}" for col in sum_generic.columns]


In [55]:
aggregated_df[sum_generic.columns]=sum_generic[sum_generic.columns]

In [56]:
sums

,year,id,総計,revenue
0,2014,1145.0,1.604335e+06,3.096367e+07
1,2014,1147.0,1.652349e+08,4.370214e+09
2,2014,1149.0,3.264717e+09,1.050980e+11
3,2014,1219.0,4.840017e+07,3.186312e+08
4,2014,1231.0,1.596449e+08,2.033818e+09
...,...,...,...,...
339,2021,4413.0,1.614168e+08,1.197058e+09
340,2021,4490.0,4.594037e+09,1.767747e+11
341,2021,6250.0,1.438952e+08,1.508488e+11
342,2021,6290.0,1.627067e+08,4.950757e+10


In [57]:
data.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital', 'oral', 'id_l4', 'Approved', 'KEGG',
       'Ingredient', 'variable', 'id', 'revenue', 'elasped'],
      dtype='object')

In [58]:
merged_df=pd.merge(data,aggregated_df,on=["year","id"])

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\reshape\merge.py:1214: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


In [59]:
merged_df.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital', 'oral', 'id_l4', 'Approved', 'KEGG',
       'Ingredient', 'variable', 'id', 'revenue', 'elasped', 'elasped_m6',
       'elasped_m5', 'elasped_m4', 'elasped_m3', 'elasped_m2', 'elasped_m1',
       'elasped_0', 'elasped_1', 'elasped_2', 'elasped_3', 'elasped_4',
       'elasped_5', 'elasped_6', 'elasped_7', 'elasped_8', 'elasped_9',
       'elasped_10', 'elasped_11', 'elasped_12', 'elasped_13', 'elasped_14',
       'elasped_15', 'elasped_16', 'elasped_17', 'elasped_18', 'elasped_19',
       'elasped_20', 'elasped_21', 'elasped_22', 'elasped_23', 'elasped_24',
       'elasped_25', 'elasped_26', 'elasped_27', 'elasped_28', 'elasped_29',
       'elasped_30', 'elasped_31', 'elasped_32', 'elasped_33', 'elasped_34',
       'elasped_35', 'elasped_36', 'elasped_37', 'elasped_38', 'sum_quantity',
       'sum_revenue', 'mean_price', 'lag_mean_price', 'lag_sum_quantity',
       'revenue0', 

connect generic usage

In [60]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [61]:
merged_df=merged_df.merge(dusage,left_on="year",right_on="Year")
merged_df.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,sum_revenue,mean_price,lag_mean_price,lag_sum_quantity,revenue0,revenue1,総計0,総計1,Year,Percentage
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,3.096367e+07,19.3,NaN,NaN,3.096367e+07,NaN,1604335.45,NaN,2014,51.55
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,3.096367e+07,19.3,NaN,NaN,3.096367e+07,NaN,1604335.45,NaN,2014,51.55
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,3.096367e+07,19.3,NaN,NaN,3.096367e+07,NaN,1604335.45,NaN,2014,51.55
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,3.096367e+07,19.3,NaN,NaN,3.096367e+07,NaN,1604335.45,NaN,2014,51.55
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,3.096367e+07,19.3,NaN,NaN,3.096367e+07,NaN,1604335.45,NaN,2014,51.55


In [62]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies["q_share"]=df_with_dummies["総計"]/df_with_dummies["sum_quantity"]
df_with_dummies["r_share"]=df_with_dummies["revenue"]/df_with_dummies["sum_revenue"]
# aggregated generic share in market i (id len 4)
df_with_dummies["generic_share_q"]=df_with_dummies["総計1"]/df_with_dummies["sum_quantity"]
df_with_dummies["generic_share_r"]=df_with_dummies["revenue1"]/df_with_dummies["sum_revenue"]

df_with_dummies.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0.182126,0.182126,NaN,NaN
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0.080302,0.080302,NaN,NaN
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0.066275,0.066275,NaN,NaN
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0.410299,0.410299,NaN,NaN
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0.260997,0.260997,NaN,NaN


In [63]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
if not(count):
    # dummy ver
    df_with_dummies.to_csv("../merged/len4_ndb_agg_blp_DropNever.csv",encoding="shift-jis")
else:
    # count ver
    df_with_dummies.to_csv("../merged/len4_ndb_sum_agg_blp_DropNever.csv",encoding="shift-jis")

In [64]:
df_with_dummies.shape

(29653, 122)